In [1]:
from pydantic import BaseModel

In [2]:
class Person(BaseModel):
    first_name:str
    last_name:str
    age:int

In [3]:
p = Person(first_name='Isaac', last_name='Newton', age=84)

In [4]:
p

Person(first_name='Isaac', last_name='Newton', age=84)

In [5]:
Person(first_name='100', last_name='100', age='3') # implicit type casting of int

Person(first_name='100', last_name='100', age=3)

In [6]:
from pydantic import ValidationError

In [7]:
try:
    Person(first_name='issac')
except ValidationError as ex:
    print(ex)

2 validation errors for Person
last_name
  Field required [type=missing, input_value={'first_name': 'issac'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/missing
age
  Field required [type=missing, input_value={'first_name': 'issac'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/missing


In [8]:
try:
    Person(first_name='issac')
except ValidationError as ex:
    print(ex.json(indent=4)) # error in json format

[
    {
        "type": "missing",
        "loc": [
            "last_name"
        ],
        "msg": "Field required",
        "input": {
            "first_name": "issac"
        },
        "url": "https://errors.pydantic.dev/2.5/v/missing"
    },
    {
        "type": "missing",
        "loc": [
            "age"
        ],
        "msg": "Field required",
        "input": {
            "first_name": "issac"
        },
        "url": "https://errors.pydantic.dev/2.5/v/missing"
    }
]


In [9]:
class Person(BaseModel):
    first_name:str
    last_name:str
    age:int | None # age is optional, None if missing # this doesn't work anymore

In [10]:
 # This doesn't work anymore
try:
    Person(first_name='Isaac', last_name='Newton')
except ValidationError as e:
    print(e)

1 validation error for Person
age
  Field required [type=missing, input_value={'first_name': 'Isaac', 'last_name': 'Newton'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/missing


In [11]:
from typing import Optional

In [12]:
class Person(BaseModel):
    first_name:str
    last_name:str
    age: Optional[int] # this too doesn't work anymore

In [13]:
# this doesn't work anymore, in latest version of pydantic
try:
    Person(first_name='Isaac', last_name='Newton')
except ValidationError as e:
    print(e)

1 validation error for Person
age
  Field required [type=missing, input_value={'first_name': 'Isaac', 'last_name': 'Newton'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/missing


In [14]:
# specify default values
class Person(BaseModel):
    first_name:str = 'Isaac' # it works
    last_name:str
    age: int = 18

In [15]:
try:
    print(Person(last_name='Newton'))
except ValidationError as e:
    print(e)

first_name='Isaac' last_name='Newton' age=18


In [16]:
# p.dict() - deprecated
p.model_dump() 

{'first_name': 'Isaac', 'last_name': 'Newton', 'age': 84}

In [17]:
# p.json()- deprecated
p.model_dump_json() # convert to json format

'{"first_name":"Isaac","last_name":"Newton","age":84}'

In [18]:
print(p.model_dump_json(exclude={'first_name'}, indent=4)) 
# exclude or include fields when serializing objects
# serialize - convert objects to json or dictionary
# deserialize - convert json or dictionary to objects

{
    "last_name": "Newton",
    "age": 84
}


In [19]:
from datetime import date

In [20]:
class Person(BaseModel):
    first_name:str = None
    last_name:str
    dob: date

In [21]:
data = {
    "first_name":"Isaac",
    "last_name":"Newton",
    "dob": date(1643, 1, 4)
}

In [22]:
# p = Person.parse_obj() - deprecated
p = Person.model_validate(data) # dictionary to the person object > deserialization

In [23]:
p

Person(first_name='Isaac', last_name='Newton', dob=datetime.date(1643, 1, 4))

In [24]:
data = {
    "first_name":"Isaac",
    "last_name":"Newton",
    "dob": "1643-01-04"
}

In [25]:
p = Person.model_validate(data) # Person class can deserialize date strings and create date objects

p

Person(first_name='Isaac', last_name='Newton', dob=datetime.date(1643, 1, 4))

In [26]:
json = '''{
    "first_name":"Isaac",
    "last_name":"Newton",
    "dob": "1643-01-04"
}'''

In [27]:
# p = Person.parse_raw() - deprecated
p = Person.model_validate_json(json)
p

Person(first_name='Isaac', last_name='Newton', dob=datetime.date(1643, 1, 4))

In [28]:
from pydantic import Field

In [29]:
# alias in pydantic
class Person(BaseModel):
    first_name:str = Field(default=None, alias='firstName')
    last_name:str = Field(alias='lastName')
    dob: date

In [30]:
try:
    Person.model_validate(data)
except ValidationError as e:
    print(e.json(indent=4)) # thanks to Field

[
    {
        "type": "missing",
        "loc": [
            "lastName"
        ],
        "msg": "Field required",
        "input": {
            "first_name": "Isaac",
            "last_name": "Newton",
            "dob": "1643-01-04"
        },
        "url": "https://errors.pydantic.dev/2.5/v/missing"
    }
]


In [31]:
data # we cannot load this data as we have specified the field aliases

{'first_name': 'Isaac', 'last_name': 'Newton', 'dob': '1643-01-04'}

In [32]:
data2 = {
    "firstName":"Isaac",
    "lastName":"Newton",
    "dob": "1643-01-04"
}

In [33]:
p = Person.model_validate(data2)
p

Person(first_name='Isaac', last_name='Newton', dob=datetime.date(1643, 1, 4))

In [34]:
p.model_dump_json() # serializing data retains the field names

'{"first_name":"Isaac","last_name":"Newton","dob":"1643-01-04"}'

In [35]:
# We can configure pydantic to allow us to the field names as well as the aliases.

In [36]:
class Person(BaseModel):
    first_name:str = Field(default=None, alias='firstName')
    last_name:str = Field(alias='lastName')
    dob: date

    class Config:
        # allow_population_by_field_name = True 
        populate_by_name = True

In [37]:
try:
    print(Person.model_validate(data)) # using field names
except ValidationError as e:
    print(e.json(indent=4))

first_name='Isaac' last_name='Newton' dob=datetime.date(1643, 1, 4)


In [38]:
try:
    print(Person.model_validate(data2)) # using field aliases
except ValidationError as e:
    print(e.json(indent=4))

first_name='Isaac' last_name='Newton' dob=datetime.date(1643, 1, 4)


In [39]:
p = Person.model_validate(data2)

In [40]:
p.model_dump_json() # serializers still use field names

'{"first_name":"Isaac","last_name":"Newton","dob":"1643-01-04"}'

In [41]:
# instruct the serializers to use the field aliases as opposed to the field names
p.model_dump_json(by_alias=True)

'{"firstName":"Isaac","lastName":"Newton","dob":"1643-01-04"}'

In [42]:
# What happens if we pass field names/aliases that are not valid for the model?

data_junk = {**data, 'junk':'extra fields'}
data_junk

{'first_name': 'Isaac',
 'last_name': 'Newton',
 'dob': '1643-01-04',
 'junk': 'extra fields'}

In [43]:
Person.model_validate(data_junk)

Person(first_name='Isaac', last_name='Newton', dob=datetime.date(1643, 1, 4))

In [44]:
p = Person(**data_junk)

In [45]:
p

Person(first_name='Isaac', last_name='Newton', dob=datetime.date(1643, 1, 4))

In [46]:
hasattr(p, 'first_name')

True

In [47]:
hasattr(p, 'junk') # junk field was totally ignored

False

We can actually define how we want to handle those extra fields:
1. We can ignore them (default)
2. We can allow them and actually add them as attributes
3. We can forbid them which raise a validation error

In [48]:
# deprecated from pydantic import Extra 
class Person(BaseModel):
    first_name:str = Field(default=None, alias='firstName')
    last_name:str = Field(alias='lastName')
    dob: date

    class Config:
        # allow_population_by_field_name = True 
        populate_by_name = True
        extra = 'forbid'

In [49]:
try:
    Person(**data_junk)
except ValidationError as e:
    print(e.json(indent=4))

[
    {
        "type": "extra_forbidden",
        "loc": [
            "junk"
        ],
        "msg": "Extra inputs are not permitted",
        "input": "extra fields",
        "url": "https://errors.pydantic.dev/2.5/v/extra_forbidden"
    }
]


In [50]:
class Person(BaseModel):
    first_name:str = Field(default=None, alias='firstName')
    last_name:str = Field(alias='lastName')
    dob: date

    class Config:
        # allow_population_by_field_name = True 
        populate_by_name = True
        extra = 'allow'

In [51]:
try:
    Person(**data_junk)
except ValidationError as e:
    print(e.json(indent=4))

In [52]:
# shortcut to enable field aliases
# We will be creating a customized base model class. 
# And we'll be re-using that class, everywhere we want to implement 
# this camel casing and alias vs field names

In [53]:
def snake_to_camel_case(value:str) -> str:
    if not isinstance(value, str):
        raise ValueError("Value must be a string")
    words = value.split('_')
    value = "".join(word.title() for word in words if word)
    return f"{value[0].lower()}{value[1:]}"

In [54]:
snake_to_camel_case("first_name")

'firstName'

In [55]:
# Now we configure our CustomBaseModel to specify that our field aliases should automatically
# be generated using the pre-defined function

In [56]:
class CustomBaseModel(BaseModel):
    class Config:
        alias_generator = snake_to_camel_case
        extra = "forbid"
        populate_by_name = True

In [57]:
class Person(CustomBaseModel):
    first_name : str = None
    last_name : str
    dob : date = None

In [58]:
p = Person(first_name='Isaac', lastName='Newton')
p

Person(first_name='Isaac', last_name='Newton', dob=None)

In [59]:
try:
    Person(**data_junk)
except ValidationError as e:
    print(e)

1 validation error for Person
junk
  Extra inputs are not permitted [type=extra_forbidden, input_value='extra fields', input_type=str]
    For further information visit https://errors.pydantic.dev/2.5/v/extra_forbidden


In [60]:
from pydantic import conint # constraint integer

class Test(CustomBaseModel):
    age: conint(gt=0, le=150)

In [61]:
Test(age=10)

Test(age=10)

In [62]:
try:
    Test(age=-1)
except ValidationError as e:
    print(e)

1 validation error for Test
age
  Input should be greater than 0 [type=greater_than, input_value=-1, input_type=int]
    For further information visit https://errors.pydantic.dev/2.5/v/greater_than


In [63]:
from pydantic import constr

In [64]:
class Test(CustomBaseModel):
    first_name: str = None
    last_name: constr(strip_whitespace=True, strict=True, min_length=2)

In [65]:
try:
    print(Test(last_name='    Newton'))
except ValidationError as e:
    print(e)

first_name=None last_name='Newton'


In [66]:
try:
    print(Test(last_name=10))
except ValidationError as e:
    print(e)

1 validation error for Test
last_name
  Input should be a valid string [type=string_type, input_value=10, input_type=int]
    For further information visit https://errors.pydantic.dev/2.5/v/string_type


Custom Validators

In [67]:
# Now if pydantics own validators are not sufficient, we can define our own custom validators
from pydantic import field_validator
class Test(CustomBaseModel):
    hash_tag : str

    @field_validator('hash_tag') # it is a class method, first argument is bound to the class
    @classmethod
    def validate_hash_tag(cls, value:str) -> str:
        if not value.startswith("#"):
            raise ValueError("Hashtag must start with a #")
        return value.lower()

In [68]:
t = Test(hash_tag='#TEST')
t

Test(hash_tag='#test')

In [69]:
try:
    Test(hash_tag='test')
except ValidationError as e:
    print(e)

1 validation error for Test
hash_tag
  Value error, Hashtag must start with a # [type=value_error, input_value='test', input_type=str]
    For further information visit https://errors.pydantic.dev/2.5/v/value_error


Autocorrecting Fields values using Custom Validators

In [70]:
class Test(CustomBaseModel):
    hash_tag: constr(min_length=5, strip_whitespace=True)

    # it is a class method, first argument is bound to the class
    @field_validator('hash_tag')
    @classmethod
    def validate_hash_tag(cls, value: str) -> str:
        if not value.startswith("#"):
            # raise ValueError("Hashtag must start with a #")
            return f"#{value.lower()}"
        return value.lower()

In [71]:
Test(hash_tag='#TEST')

Test(hash_tag='#test')

In [72]:
try:
    print(Test(hash_tag='TEST2'))
except ValidationError as e:
    print(e)

hash_tag='#test2'


Multi-Field Custom Validators

In [73]:
from enum import Enum
from typing import List, Tuple, Union

In [74]:
class PolygonType(Enum):
    triangle = 3
    tetragon = 4
    pentagon = 5
    hexagon = 6

In [75]:
t = PolygonType.triangle

In [76]:
t.value

3

In [77]:
t.name

'triangle'

In [78]:
class Polygon(CustomBaseModel):
    polygon_type: PolygonType
    edges: List[Tuple[int|float, int|float]]

    # this validator is gonna run on vertices, whether validators on polygon_type have passed or not
    @field_validator('edges')
    @classmethod
    def validate_edges(cls, value, values):
        # value > value for vertices which is gonna be a list
        # values > A dictionary that contains all the fields that were validated successfully listed above the current field. If it didn't pass the validation, it won't be present in values
        # print(values)
        polygon_type = values.data.get('polygon_type') # ValidationInfo object
        if polygon_type:
            num_vertices_required = polygon_type.value
            if len(value) != num_vertices_required:
                raise ValueError(
                    f"For a {polygon_type.name}, exactly {polygon_type.value}"
                    "vertices are required."
                )
        return value
    

In [79]:
Polygon(polygonType=PolygonType.triangle, edges=[(1,2),(2,3),(3,1)])

Polygon(polygon_type=<PolygonType.triangle: 3>, edges=[(1, 2), (2, 3), (3, 1)])

In [80]:
try:
    Polygon(polygonType=PolygonType.triangle, edges=[(1, 2),(2,3+1j),(3,1)])
except ValidationError as e:
    print(e.json(indent=4))

[
    {
        "type": "int_type",
        "loc": [
            "edges",
            1,
            1,
            "int"
        ],
        "msg": "Input should be a valid integer",
        "input": "(3+1j)",
        "url": "https://errors.pydantic.dev/2.5/v/int_type"
    },
    {
        "type": "float_type",
        "loc": [
            "edges",
            1,
            1,
            "float"
        ],
        "msg": "Input should be a valid number",
        "input": "(3+1j)",
        "url": "https://errors.pydantic.dev/2.5/v/float_type"
    }
]


In [81]:
try:
    Polygon(polygonType=PolygonType.pentagon,
            edges=[(1, 2), (2, 3), (3, 1)])
except ValidationError as e:
    print(e)

1 validation error for Polygon
edges
  Value error, For a pentagon, exactly 5vertices are required. [type=value_error, input_value=[(1, 2), (2, 3), (3, 1)], input_type=list]
    For further information visit https://errors.pydantic.dev/2.5/v/value_error


In [82]:
try:
    Polygon(polygonType=10,
            edges=[(1, 2), (2, 3), (3, 1)])
except ValidationError as e:
    print(e)

1 validation error for Polygon
polygonType
  Input should be 3, 4, 5 or 6 [type=enum, input_value=10, input_type=int]


Nested Models using Model Composition

* post
    * byline (one or more authors)
        * author:
            * first_name (required, min 2 chars, max 20 chars)
            * last_name (required, min 1 char, max 20 chars)
            * display_name (optional, default to first name, initial of last name, min 1 char, max 25 chars)
    * title (required, at least 10 characters, no more than 50, force title case)
    * sub title (optional, if present at least 20 characters, max 100)
    * body (required, at least 100 characters, no upper limit)
    * links (0 or more)
        * link:
            * name (required, min 5 characters, max 25 characters)
            * url (required, valid url, that must include scheme (http/https))

In [83]:
from typing_extensions import Annotated, Any
from pydantic import StringConstraints

In [84]:
# Author Class
class Author(CustomBaseModel):
    first_name: constr(min_length=2, max_length=20, strip_whitespace=True)
    last_name: constr(min_length=1, max_length=20, strip_whitespace=True)
    # display_name: constr(min_length=5, max_length=25) = Field(default=None,
    #     validate_default=True)
    display_name: Annotated[str, StringConstraints(min_length=4, max_length=25)] = Field(default=str(None), validate_default=True)

    # always = True forces the validator to run, even if the display is None,
    # this is how we can set dynamic default value

    @field_validator("display_name")
    @classmethod
    def validate_display_name(cls, value, values):
        # validator runs, even if previous did not validate properly - so
        # we need to run our code only if prior fields validated OK
        # print(value)
        if value == 'None':
            if 'first_name' in values.data and 'last_name' in values.data:
                first_name = values.data.get('first_name')
                last_name = values.data.get('last_name')
                return f"{first_name} {(last_name[0]).upper()}"
        
        return value

In [85]:
try:
    print(Author(first_name='Gottfried', last_name='Leibniz'))
except ValidationError as e:
    print(e)

first_name='Gottfried' last_name='Leibniz' display_name='Gottfried L'


In [86]:
try:
    print(Author(first_name='Gottfried', last_name='Leibniz', displayName='Johnny'))
except ValidationError as e:
    print(e)

first_name='Gottfried' last_name='Leibniz' display_name='Johnny'


In [87]:
try:
    print(Author(first_name='Gottfried', last_name='Leibniz'*10, displayName='Johnny'))
except ValidationError as e:
    print(e)

1 validation error for Author
last_name
  String should have at most 20 characters [type=string_too_long, input_value='LeibnizLeibnizLeibnizLei...izLeibnizLeibnizLeibniz', input_type=str]
    For further information visit https://errors.pydantic.dev/2.5/v/string_too_long


In [88]:
from pydantic import AnyHttpUrl

In [89]:
class Link(CustomBaseModel):
    name: Annotated[str, StringConstraints(min_length=5, max_length=100)]
    url: AnyHttpUrl

In [90]:
try:
    print(Link(name="google",url="https://www.google.com"))
except ValidationError as e:
    print(e)

name='google' url=Url('https://www.google.com/')


In [91]:
try:
    print(Link(name="google", url="www.google.com"))
except ValidationError as e:
    print(e)

1 validation error for Link
url
  Input should be a valid URL, relative URL without a base [type=url_parsing, input_value='www.google.com', input_type=str]
    For further information visit https://errors.pydantic.dev/2.5/v/url_parsing


In [92]:
from pydantic import conlist

In [93]:
class Post(CustomBaseModel):
    byline: conlist(item_type=Author, min_length=1)
    title: Annotated[str, StringConstraints(min_length=10,max_length=50, strip_whitespace=True)]

    @field_validator('title')
    @classmethod
    def validate_title(cls, value):
        return value and value.title()

In [94]:
Post(
    byline=[
        Author(first_name='Isaac', lastName='Newton')
    ],
    title = "Mathmatica Principia"
)

Post(byline=[Author(first_name='Isaac', last_name='Newton', display_name='Isaac N')], title='Mathmatica Principia')

In [95]:
class Post(CustomBaseModel):
    byline: conlist(item_type=Author, min_length=1)
    title: Annotated[str, StringConstraints(
        min_length=10, max_length=50, strip_whitespace=True)]
    sub_title: Annotated[str, StringConstraints(
        min_length=20, max_length=100, strip_whitespace=True
    )] = 'None'
    body: Annotated[str, StringConstraints(min_length=100)]

    @field_validator('title')
    @classmethod
    def validate_title(cls, value):
        return value and value.title()

In [96]:
print(
    Post(
        byline=[
            Author(first_name='Isaac', lastName='Newton')
        ],
        title="Mathmatica Principia",
        sub_title="Laws governing everyday physics",
        body="*" * 101
    ).model_dump_json(indent=4)
)

{
    "byline": [
        {
            "first_name": "Isaac",
            "last_name": "Newton",
            "display_name": "Isaac N"
        }
    ],
    "title": "Mathmatica Principia",
    "sub_title": "Laws governing everyday physics",
    "body": "*****************************************************************************************************"
}


In [97]:
class Post(CustomBaseModel):
    byline: conlist(item_type=Author, min_length=1)

    title: Annotated[str, StringConstraints(
        min_length=10, max_length=50, strip_whitespace=True)]
    
    sub_title: Annotated[str, StringConstraints(
        min_length=20, max_length=100, strip_whitespace=True
    )] = 'None'

    body: Annotated[str, StringConstraints(min_length=100)]
    
    links: conlist(item_type=Link, min_length=0)

    @field_validator('title')
    @classmethod
    def validate_title(cls, value):
        return value and value.title()

In [98]:
print(
    Post(
        byline=[
            Author(first_name='Isaac', lastName='Newton')
        ],
        title="Mathmatica Principia",
        sub_title="Laws governing everyday physics",
        body="*" * 101,
        links=[
            Link(name="Newton\’s Philosophiae Naturalis Principia Mathematica",url="https://plato.stanford.edu/entries/newton-principia/")
        ]
    ).model_dump_json(indent=4)
)

{
    "byline": [
        {
            "first_name": "Isaac",
            "last_name": "Newton",
            "display_name": "Isaac N"
        }
    ],
    "title": "Mathmatica Principia",
    "sub_title": "Laws governing everyday physics",
    "body": "*****************************************************************************************************",
    "links": [
        {
            "name": "Newton\\’s Philosophiae Naturalis Principia Mathematica",
            "url": "https://plato.stanford.edu/entries/newton-principia/"
        }
    ]
}


In [99]:
post = Post(
        byline=[
            Author(first_name='Isaac', lastName='Newton')
        ],
        title="Mathmatica Principia",
        sub_title="Laws governing everyday physics",
        body='''Lorem ipsum dolor sit amet, consectetur adipiscing elit. In ac pharetra ex, in rutrum neque. 
        Mauris lacinia leo vitae purus ultricies venenatis. Etiam eu egestas enim, id mattis sapien. 
        Fusce nec fermentum dolor. Nam dolor nulla, tristique quis semper in, placerat at ligula. Nam vel 
        lectus quis turpis feugiat efficitur. Aenean suscipit dignissim ante, ac tempus lacus iaculis non. 
        Morbi in condimentum turpis. Praesent erat lorem, tempus a bibendum a, pharetra eu ex. Integer posuere,
        purus placerat maximus molestie, nisi nisl molestie arcu, eget viverra quam enim nec augue. Ut ut eros 
        velit. Donec felis orci, semper nec velit in, varius consectetur velit. Integer sodales a ante ut blandit.
        Vestibulum diam tellus, sollicitudin id volutpat vehicula, ultricies id erat.''',
        links=[
            Link(name="Newton\’s Philosophiae Naturalis Principia Mathematica",
                 url="https://plato.stanford.edu/entries/newton-principia/")
        ]
    )

from pprint import pprint
print(post.model_dump_json(indent=4))

{
    "byline": [
        {
            "first_name": "Isaac",
            "last_name": "Newton",
            "display_name": "Isaac N"
        }
    ],
    "title": "Mathmatica Principia",
    "sub_title": "Laws governing everyday physics",
    "body": "Lorem ipsum dolor sit amet, consectetur adipiscing elit. In ac pharetra ex, in rutrum neque. \n        Mauris lacinia leo vitae purus ultricies venenatis. Etiam eu egestas enim, id mattis sapien. \n        Fusce nec fermentum dolor. Nam dolor nulla, tristique quis semper in, placerat at ligula. Nam vel \n        lectus quis turpis feugiat efficitur. Aenean suscipit dignissim ante, ac tempus lacus iaculis non. \n        Morbi in condimentum turpis. Praesent erat lorem, tempus a bibendum a, pharetra eu ex. Integer posuere,\n        purus placerat maximus molestie, nisi nisl molestie arcu, eget viverra quam enim nec augue. Ut ut eros \n        velit. Donec felis orci, semper nec velit in, varius consectetur velit. Integer sodales a ante u

In [100]:
json_data = post.model_dump_json(indent=4) # deserializer

In [101]:
print((Post.model_validate_json(json_data=json_data)).model_dump_json(indent=4)) # serialize

{
    "byline": [
        {
            "first_name": "Isaac",
            "last_name": "Newton",
            "display_name": "Isaac N"
        }
    ],
    "title": "Mathmatica Principia",
    "sub_title": "Laws governing everyday physics",
    "body": "Lorem ipsum dolor sit amet, consectetur adipiscing elit. In ac pharetra ex, in rutrum neque. \n        Mauris lacinia leo vitae purus ultricies venenatis. Etiam eu egestas enim, id mattis sapien. \n        Fusce nec fermentum dolor. Nam dolor nulla, tristique quis semper in, placerat at ligula. Nam vel \n        lectus quis turpis feugiat efficitur. Aenean suscipit dignissim ante, ac tempus lacus iaculis non. \n        Morbi in condimentum turpis. Praesent erat lorem, tempus a bibendum a, pharetra eu ex. Integer posuere,\n        purus placerat maximus molestie, nisi nisl molestie arcu, eget viverra quam enim nec augue. Ut ut eros \n        velit. Donec felis orci, semper nec velit in, varius consectetur velit. Integer sodales a ante u

In [102]:
print(post.model_dump_json(by_alias=True, indent=2))

{
  "byline": [
    {
      "firstName": "Isaac",
      "lastName": "Newton",
      "displayName": "Isaac N"
    }
  ],
  "title": "Mathmatica Principia",
  "subTitle": "Laws governing everyday physics",
  "body": "Lorem ipsum dolor sit amet, consectetur adipiscing elit. In ac pharetra ex, in rutrum neque. \n        Mauris lacinia leo vitae purus ultricies venenatis. Etiam eu egestas enim, id mattis sapien. \n        Fusce nec fermentum dolor. Nam dolor nulla, tristique quis semper in, placerat at ligula. Nam vel \n        lectus quis turpis feugiat efficitur. Aenean suscipit dignissim ante, ac tempus lacus iaculis non. \n        Morbi in condimentum turpis. Praesent erat lorem, tempus a bibendum a, pharetra eu ex. Integer posuere,\n        purus placerat maximus molestie, nisi nisl molestie arcu, eget viverra quam enim nec augue. Ut ut eros \n        velit. Donec felis orci, semper nec velit in, varius consectetur velit. Integer sodales a ante ut blandit.\n        Vestibulum diam tell

In [103]:
# You can ask pydantic to give you the JSON schema for the Post model
import json
print(json.dumps(Post.model_json_schema(), indent=4))

{
    "$defs": {
        "Author": {
            "additionalProperties": false,
            "properties": {
                "firstName": {
                    "maxLength": 20,
                    "minLength": 2,
                    "title": "Firstname",
                    "type": "string"
                },
                "lastName": {
                    "maxLength": 20,
                    "minLength": 1,
                    "title": "Lastname",
                    "type": "string"
                },
                "displayName": {
                    "default": "None",
                    "maxLength": 25,
                    "minLength": 4,
                    "title": "Displayname",
                    "type": "string"
                }
            },
            "required": [
                "firstName",
                "lastName"
            ],
            "title": "Author",
            "type": "object"
        },
        "Link": {
            "additionalProperties": false,
  

### Handson

* Book
    * authors: a list of Author(s)
        * Author:
            * name
    * title: string: Title of the book 
    * desc: string: Description of the book
    * genre: strictly string value, List of genres a book can be classified into
    * isbn: a string of length 7-10, alphanumeric
    * pages: number of pages the book has
    * rating: Book rating
    * bookformat: 

In [104]:
from pydantic import StrictStr, StrictInt, StrictFloat

In [105]:
from enum import Enum

class BookFormat(Enum):
    Paperback = 0
    Hardcover = 1

In [106]:
class Book(BaseModel):
    title: Annotated[str, StringConstraints(
        strip_whitespace=True, strict=True)]
    
    authors: conlist(item_type=str, min_length=1) = Field(alias='author')

    description: Annotated[str, StringConstraints(
        strip_whitespace=True, strict=True)] = Field(alias='desc')
    
    genres: conlist(item_type=str, min_length=1) = Field(alias='genre')

    book_format: str = Field(default='NA', validate_default=True,alias='bookformat')

    isbn: Annotated[str, StringConstraints(
        strip_whitespace=True, strict=True)]
    
    pages: StrictInt

    rating: StrictFloat

    @field_validator('book_format')
    @classmethod
    def validate_book_format(cls, value):
        if value == "Hardcover":
            return BookFormat.Hardcover.name
        elif value == "Paperback":
            return BookFormat.Paperback.name
        else:
            return value
        
    @field_validator('pages')
    @classmethod
    def validate_pages(cls, value):
        if value < 50:
            raise ValueError("Number of pages must be more than 100 for a book")
        return value

    class Config:
        populate_by_name = True
        extra = 'forbid'
        


In [107]:
import os
print(os.listdir())

['basics.py', 'main.py', 'output.json', 'pydantic-tutorial.ipynb']


In [108]:
with open('output.json','r') as f:
    json_data = json.load(f)
    

In [109]:
try:
    print(Book(**json_data[10]).model_dump())
except ValidationError as e:
    print(e)

{'title': 'Happiness: Lessons from a New Science', 'authors': ['Richard Layard'], 'description': "There is a paradox at the heart of our lives. We all want more money, but as societies become richer, they do not become happier. This is not speculation: It's the story told by countless pieces of scientific research. We now have sophisticated ways of measuring how happy people are, and all the evidence shows that on average people have grown no happier in the last fifty years, even as average incomes have more than doubled.The central question the great economist Richard Layard asks in ,Happiness, is this: If we really wanted to be happier, what would we do differently? First we'd have to see clearly what conditions generate happiness and then bend all our efforts toward producing them. That is what this book is about-the causes of happiness and the means we have to effect it.,Until recently there was too little evidence to give a good answer to this essential question, but, Layard shows

In [110]:
import random

def get_book_index(length):
    return random.choice(range(0,length))


In [111]:
try:
    print(Book(**json_data[10]).model_dump())
except ValidationError as e:
    print(e)

{'title': 'Happiness: Lessons from a New Science', 'authors': ['Richard Layard'], 'description': "There is a paradox at the heart of our lives. We all want more money, but as societies become richer, they do not become happier. This is not speculation: It's the story told by countless pieces of scientific research. We now have sophisticated ways of measuring how happy people are, and all the evidence shows that on average people have grown no happier in the last fifty years, even as average incomes have more than doubled.The central question the great economist Richard Layard asks in ,Happiness, is this: If we really wanted to be happier, what would we do differently? First we'd have to see clearly what conditions generate happiness and then bend all our efforts toward producing them. That is what this book is about-the causes of happiness and the means we have to effect it.,Until recently there was too little evidence to give a good answer to this essential question, but, Layard shows

In [112]:
for i in range(5):
    try:
        data = json_data[get_book_index(len(json_data))]
        print(Book(**data).model_dump_json(indent=4))
    except ValidationError as e:
        print(e)

{
    "title": "Web Design on a Shoestring",
    "authors": [
        "Carrie Bickner"
    ],
    "description": "Providing do-it-yourself, budget-conscious ideas for web site design and development, this text includes tips and techniques, examining before and after shots of existing web sites.",
    "genres": [
        "Web",
        "Design",
        "Internet",
        "Website Design"
    ],
    "book_format": "Paperback",
    "isbn": "735713286",
    "pages": 215,
    "rating": 2.97
}
{
    "title": "Paul: His Letters and His Theology: An Introduction to Paul's Epistles",
    "authors": [
        "Stanley B. Marrow"
    ],
    "description": "A major Pauline theology, the first to have come out in the Catholic area in recent years, which sheds light on and interprets Paul's theology by his letters, his life, and both against the background of his times.",
    "genres": [
        "Religion",
        "Christianity",
        "Theology"
    ],
    "book_format": "Paperback",
    "isbn